#### 제목: 알라딘 Open API를 사용하여 원하는 카테고리의 도서 목록을 불러와 임베딩하기 

##### 목적: 알라딘 api를 사용해 원하는 카테고리에 속하는 도서들의 제목, 작가, 목차만 모아서 임베딩한 후, RAG 시스템에 활용한다.

##### 사용 프로그램
- FAISS DB
- BeautifulSoup
<br/>
<br/>

##### 수집하는 책의 범위
- 국내 도서
<br/>
<br/>

##### 카테고리
- 고등학교참고서
- 수험서/자격증
- 외국어
- 중학교참고서
- 초등학교참고서
- 컴퓨터/모바일
<br/>
<br/>

##### 알라딘 api 사용 
- 하루 최대 5000개 요청
<br/>
<br/>

---
<br/>

> 참고: [알라딘 OpenAPI 메뉴얼](https://docs.google.com/document/d/1mX-WxuoGs8Hy-QalhHcvuV17n50uGI2Sg_GHofgiePE/edit?tab=t.0#)

> 참고: [알라딘 모든 분야 카테고리](https://image.aladin.co.kr/img/files/aladin_Category_CID_20210927.xls)

In [1]:
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from typing import List, Dict
import pandas as pd
import numpy as np
import requests
import pickle
import faiss
import json
import os

In [2]:
load_dotenv()

ALADIN_API_KEY = os.getenv("ALADIN_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_EMBEDDING_MODEL = os.getenv("OPENAI_EMBEDDING_MODEL")

In [3]:
def get_cid():
    """
    알라딘 카테고리 ID를 담은 csv 파일을 불러와 원하는 분야만 필터링하는 함수 
    """
    cid = pd.read_csv("aladin_CID.csv", dtype={"CID": "Int64"})  # CID 열은 정수로 불러오기 

    cid = cid[cid["몰"]=="국내도서"]  # 국내 도서만 한정 

    # 원하는 분야들만 필터링 
    cid = cid[cid["1Depth"].isin([
        '고등학교참고서',
        '수험서/자격증',
        '외국어',
        '중학교참고서',
        '초등학교참고서',
        '컴퓨터/모바일'
    ])]
    
    return cid

def get_isbn13(c_name, c_id):
    """
    검색어를 입력하면 isbn13을 얻어오는 함수 
    알라딘 OpenAPI 메뉴얼의 상품 검색 API 사용 

    1. Request
    - ttbkey: 알라딘 API 인증 키 (필수)
    - Query: 검색어 (필수)
    - QueryType: 검색어 종류
    - SearchTarget: 검색 대상 
    - start: 검색 결과 시작 페이지 
    - MaxResults: 검색 결과 한 페이지 당 최대 출력 개수 
    - CategoryId: 특정 분야로 검색 결과 제한 
    - output: 출력 방법 

    2. Response
    - item: 상품 정보 
    """
    # 검색된 도서를 담을 리스트 생성 
    isbns = []

    # 도서 검색어에 카테고리명과 카테고리 ID 입력 
    Query = c_name
    CategoryId = c_id

    # API URL
    get_isbn_url = "http://www.aladin.co.kr/ttb/api/ItemSearch.aspx"

    # Request 정의
    isbn_params = {
        "ttbkey": ALADIN_API_KEY,
        "Query": Query,
        "QueryType": "Keyword",  # 제목&저자 검색
        "SearchTarget": "Book",
        "start": 1,
        "MaxResult": 10,
        "CategoryId": CategoryId,
        "output": "js",  # JSON 형식 
    }

    # GET 요청 
    isbn_response = requests.get(url=get_isbn_url, params=isbn_params)

    # 응답 확인 
    if isbn_response.status_code == 200:
        # json으로 변환 
        json_data = json.loads(rf"{isbn_response.text[:-1]}".replace('\\', '\\\\'))  # 마지막에 ;를 빼기 위함 
        
        # 도서 정보 추출 
        json_data_items = json_data["item"]
        if len(json_data_items) == 0:
            pass
        else:
            for item in json_data_items:
                isbns.append(item)
    else:
        raise Exception(f"isbn 요청 실패: {isbn_response.status_code}")

    return isbns

def get_books(isbns):
    """
    검색된 isbn13으로 책의 제목과 목차를 얻어오는 함수 
    알라딘 OpenAPI 메뉴얼의 상품 검색 API 사용 

    1. Request
    - ttbkey: 알라딘 API 인증 키 (필수)
    - ItemId: 상품을 구분짓는 유일한 값 (필수)
    - ItemIdType: ItemId가 ISBN으로 입력됐는지, 알라딘 고유의 ItemId인지 선택 
    - Output: 출력 방법 
    - OptResult: [Toc, categoryIdList] (목차, 전체 분야)

    2. Response
    - item: 상품 정보 
    """
    # API URL
    get_toc_url = "http://www.aladin.co.kr/ttb/api/ItemLookUp.aspx"

    # 검색된 도서의 목차들을 담을 리스트 생성  
    tocs = []

    for isbn in isbns:
        # Request 정의
        toc_params = {
            "ttbkey": ALADIN_API_KEY,
            "ItemId": isbn["isbn13"],
            "ItemIdType": "ISBN13",
            "Output": "js",  # JSON 형식 
            "OptResult": ["Toc", "categoryIdList"]
        }

        # GET 요청 
        toc_response = requests.get(url=get_toc_url, params=toc_params)

        # 응답 확인 
        if toc_response.status_code == 200:
            try:
                # json으로 변환
                json_data = json.loads(rf"{toc_response.text[:-1]}".replace('\\', '\\\\'))["item"][0]
                
                # 목차 추출 
                json_data_item = {
                    'title': json_data['title'],
                    'author': json_data['author'],
                    'pubDate': json_data['pubDate'],
                    'description': json_data['description'],
                    'categoryName': json_data['categoryName'],
                    'toc': json_data['bookinfo']['toc'],
                }
                tocs.append(json_data_item)
            except Exception:
                continue

        else:
            raise Exception(f"목차 요청 실패: {toc_response.status_code}")

    return tocs

def save_to_json(start: int, k:int):
    """데이터를 json 파일로 저장하는 함수"""
    dir_path = "./books/"
    cid = get_cid()
    count = 0
    
    for i in range(k):
        try:
            c_name = cid["카테고리명"].iloc[start+i]
            c_id = cid["CID"].iloc[start+i]
            isbns = get_isbn13(c_name=c_name, c_id=c_id)
            books = get_books(isbns)
            
            if len(books) == 0:
                continue
            
            count += len(books)
            c_name = c_name.replace('/', '_')
            with open(f'{dir_path}/{c_name}.json', 'w', encoding='utf-8') as f:
                json.dump(books, f, ensure_ascii=False, indent=4)
        except Exception as e:
            continue
        
        print('='*30)
        print(f"{start+i}번째 행 검색 중..")
        print(f"{c_name} 카테고리에서 {len(books)}개의 책을 찾았습니다.\n")
        
    print(f"총 {count}개의 책을 찾았습니다.")

In [4]:
cid = get_cid()
cid

,CID,카테고리명,몰,1Depth,2Depth,3Depth,4Depth,5Depth,Unnamed: 8,Unnamed: 9
171,76001,고등학교참고서,국내도서,고등학교참고서,NaN,NaN,NaN,NaN,NaN,NaN
172,77021,고등-문제집,국내도서,고등학교참고서,고등-문제집,NaN,NaN,NaN,NaN,NaN
173,77129,과학탐구,국내도서,고등학교참고서,고등-문제집,과학탐구,NaN,NaN,NaN,NaN
174,77125,국어영역,국내도서,고등학교참고서,고등-문제집,국어영역,NaN,NaN,NaN,NaN
175,77130,기타영역,국내도서,고등학교참고서,고등-문제집,기타영역,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4776,6889,디지털 카메라,국내도서,컴퓨터/모바일,PC/게임/디지털 카메라,디지털 카메라,NaN,NaN,NaN,NaN
4777,6890,디지털 캠코더,국내도서,컴퓨터/모바일,PC/게임/디지털 카메라,디지털 캠코더,NaN,NaN,NaN,NaN
4778,2615,인터넷/윈도우즈 배우기,국내도서,컴퓨터/모바일,PC/게임/디지털 카메라,인터넷/윈도우즈 배우기,NaN,NaN,NaN,NaN
4779,3023,초보자를 위한 컴퓨터 책,국내도서,컴퓨터/모바일,PC/게임/디지털 카메라,초보자를 위한 컴퓨터 책,NaN,NaN,NaN,NaN


In [5]:
save_to_json(1027, 10)

1028번째 행 검색 중..
취업 정보 카테고리에서 3개의 책을 찾았습니다.

1029번째 행 검색 중..
컴퓨터 활용능력 카테고리에서 10개의 책을 찾았습니다.

1030번째 행 검색 중..
사무자동화 카테고리에서 10개의 책을 찾았습니다.

1032번째 행 검색 중..
워드프로세서 카테고리에서 10개의 책을 찾았습니다.

1033번째 행 검색 중..
워드프로세서 실기 카테고리에서 9개의 책을 찾았습니다.

1034번째 행 검색 중..
워드프로세서 필기 카테고리에서 10개의 책을 찾았습니다.

1035번째 행 검색 중..
웹디자인 기능사 카테고리에서 10개의 책을 찾았습니다.

1036번째 행 검색 중..
인터넷정보관리사 카테고리에서 4개의 책을 찾았습니다.

총 66개의 책을 찾았습니다.


In [6]:
count = 0
path = './books'

for _, _, files in os.walk(path):
    for file in files:
        with open(f"{path}/{file}", 'r') as f:
            json_data = json.load(f)
            count += len(json_data)

print(f"총 도서 수: {count}")

총 도서 수: 5701
